In [ ]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 진행 여부 함수
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print("실패: 'y' 혹은 'n' 를 입력해 주세요")

### 추가 1) 메타데이터 띄움처리 전 중복 제거..
### 추가 2) 한글명 notnull vs null 분리하여 notnull만 관리(null: 파일 생성 및 기입 요청)
### 추가 3.1) 한글 용어 점검 기능: 한글명+영문명+숫자조합 가능), 그외 모두 제거
### 추가 3.2) 한글 단어 점검 기능: 한글명(숫자조합 가능), 영문명만(숫자조합 가능) 그외 모두 제거
### 추가 4) 테이블기준 상세항목 정의하기.. 

In [ ]:
# 표준용어 구분자 변환 점검
print(len('가 나 다'.split(" ")))
'가 나 다'.split(" ")

In [ ]:
'가 나 다'.count(" ")

# 한글명은 notNull이어야 함

작업내용:
1. 중복제거(unique): 작업 효율성 제고
2. 개수 매칭 -> (모든 값 채우기)
3. 순서 매칭: ID 부여, 한글/영문
4. 재조립( 한글명 기준)

# 1. 파일 불러오기

In [ ]:
# 파일 불러오기
# test_emoti_dup_korDup
sheet_name = "input_test"
input_path = os.path.join(os.getcwd(), "input\\")
output_path = os.path.join(os.getcwd(), "output\\")
# input_test = pd.read_excel(input_path+"test_emoti_dup_korDup.xlsx", sheet_name=sheet_name, header=0)
input_test = pd.read_excel(input_path+"test.xlsx", sheet_name=sheet_name, header=0)
# test_final.xlsx
# null 제거
for i in range(len(input_test.columns)):
    input_test[input_test.columns[i]] = input_test[input_test.columns[i]].fillna("")
print(input_test.shape)
input_test.head(5)

# 2. 용어 분리

In [ ]:
# 인덱스 DF 초기화
df_index = pd.DataFrame(input_test['No'])
df_index.columns = ['index']

In [ ]:
# 용어명 분리

# 용어명 분리
input_split_kor = pd.DataFrame(input_test['용어명 분리'].str.split('_').tolist()).fillna("")

# 영문명 없을 경우 대비하여 cnt 집계
kor_firts_cnt = len(input_split_kor.columns)

# 컬럼명 입력
col_list_kor = []
for i in range(len(input_split_kor.columns)):
    col_list_kor.append("stwd_kor{}".format(i))
input_split_kor.columns = col_list_kor

# 한글 개수 세기
dupl_list_kor = []
dupl_cnt_kor = []
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = []
    for j in range(len(input_split_kor["stwd_kor{}".format(i)])):
        dupl_list_kor = input_split_kor["stwd_kor{}".format(i)].to_list().count(input_split_kor["stwd_kor{}".format(i)][j])
        globals()["dupl_list_kor{}".format(i)].append(dupl_list_kor)
for i in range(kor_firts_cnt):
    globals()["dupl_list_kor{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_kor{}".format(i)])], axis = 0)
    globals()["dupl_list_kor{}".format(i)].columns = ["dupl_list_kor{}".format(i)]
    
# 컬럼명,index, 단어위치 추가
for i in range(len(input_split_kor.columns)):
    input_split_kor["num_kor{}".format(i)] = "num_kor{}".format(i)
    globals()["stwd_kor{}".format(i)] = pd.concat(
            [
            df_index.fillna(""), 
            pd.DataFrame(input_split_kor["stwd_kor{}".format(i)]).fillna(""),
            pd.DataFrame(input_split_kor["num_kor{}".format(i)]).fillna(""),
            globals()["dupl_list_kor{}".format(i)].fillna("")
            ], 
            axis=1)
print(col_list_kor)
print(kor_firts_cnt)
# stwd_kor9
# dupl_list_kor0

In [ ]:
# 영문명 분리

# 영문명이 하나라도 입력돼 있을 경우
if len(input_test['영문약어명']) > 0:
    print("영문명 컬럼 수 1개 이상:"+str((input_test['영문약어명']=="").sum()))

    # 영문명 분리
    input_split_eng = pd.DataFrame(input_test['영문약어명'].str.split('_').tolist()).fillna("")
    
    # 영문명 컬럼 수
    eng_firts_cnt = len(input_split_eng.columns)
    
    # 컬럼명 입력
    col_list_eng = []
    for i in range(len(input_split_eng.columns)):
        col_list_eng.append("stwd_eng{}".format(i))
    input_split_eng.columns = col_list_eng
    
    # 영문 개수 세기
    dupl_list_eng = []
    dupl_cnt_eng = []
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = []
        for j in range(len(input_split_eng["stwd_eng{}".format(i)])):
            dupl_list_eng = input_split_eng["stwd_eng{}".format(i)].to_list().count(input_split_eng["stwd_eng{}".format(i)][j])
            globals()["dupl_list_eng{}".format(i)].append(dupl_list_eng)
    for i in range(eng_firts_cnt):
        globals()["dupl_list_eng{}".format(i)] = pd.concat([pd.DataFrame(globals()["dupl_list_eng{}".format(i)])], axis = 0)
        globals()["dupl_list_eng{}".format(i)].columns = ["dupl_list_eng{}".format(i)]
        
    # 컬럼명,index, 단어위치 추가
    for i in range(eng_firts_cnt):
        input_split_eng["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)] = pd.concat(
                [
                df_index.fillna(""), 
                pd.DataFrame(input_split_eng["stwd_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                globals()["dupl_list_eng{}".format(i)].fillna("")
                ], 
                axis=1)
        
# 영문명이 없을 경우
elif len(input_test['영문약어명']) == 0:
    print("영문명 컬럼 수 0개:"+str((input_test['영문약어명']=="").sum()))
    
    col_list_eng = []
    for i in range(kor_firts_cnt):
        globals()["stwd_eng{}".format(i)] = pd.DataFrame(columns = ["stwd_eng{}".format(i)])
        globals()["num_eng{}".format(i)] = pd.DataFrame(columns = ["num_eng{}".format(i)])
        globals()["dupl_list_eng{}".format(i)] = pd.DataFrame(columns = ["dupl_list_eng{}".format(i)])
        globals()["stwd_eng{}".format(i)] = pd.concat(
            [
                df_index.fillna(""), 
                globals()["stwd_eng{}".format(i)].fillna(""),
                globals()["num_eng{}".format(i)].fillna(""),
                globals()["dupl_list_eng{}".format(i)].fillna(""),
            ]
            , axis = 1
        )
        globals()["stwd_eng{}".format(i)]["num_eng{}".format(i)] = "num_eng{}".format(i)
        globals()["stwd_eng{}".format(i)]["dupl_list_eng{}".format(i)] = len(globals()["stwd_eng{}".format(i)])
        col_list_eng.append("stwd_eng{}".format(i))
        
else:
    print("한글컬럼, 영문커럼 예외 사항")

# 컬럼수 일치 점검
col_cnt_kor = len(col_list_kor)
col_cnt_eng = len(col_list_eng)
if len(col_list_kor) == len(col_list_eng):
    print("한글컬럼수와 영문컬럼수 일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
else:
    print("한글컬럼수와 영문컬럼수 불일치")
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))                                                       
print(col_list_eng)
stwd_eng0

In [ ]:
# 한글/영문 컬럼 수 일치시키기

# 한글컬럼과 영문컬럼 수 일치할 경우
if col_cnt_kor == col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = None
    col_cnt_diff = None
    print('한글컬럼과 영문컬럼 수 일치')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
    
# 한글컬럼이 더 많을 경우
elif col_cnt_kor > col_cnt_eng:
    col_cnt = col_cnt_kor
    col_cnt_diff_list = list(set(range(col_cnt_kor)) - set(range(col_cnt_eng)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_eng['stwd_eng{}'.format(i)] = None
        input_split_eng['num_eng{}'.format(i)] = None
        input_split_eng['dupl_list_eng{}'.format(i)] = None
        globals()['stwd_eng{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_eng['stwd_eng{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_eng["num_eng{}".format(i)].fillna("")),
                pd.DataFrame(input_split_eng["dupl_list_eng{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('한글컬럼이 더 많은 싱태')
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  

# 영문컬럼이 더 많을 경우
elif col_cnt_kor < col_cnt_eng:
    col_cnt = col_cnt_eng
    col_cnt_diff_list = list(set(range(col_cnt_eng)) - set(range(col_cnt_kor)))
    col_cnt_diff = len(col_cnt_diff_list)
    for i in col_cnt_diff_list:
        input_split_kor['stwd_kor{}'.format(i)] = None
        input_split_kor['num_kor{}'.format(i)] = None
        input_split_eng['dupl_list_kor{}'.format(i)] = None
        globals()['stwd_kor{}'.format(i)] = pd.concat(
            [
                df_index,
                pd.DataFrame(input_split_kor['stwd_kor{}'.format(i)].fillna("")),
                pd.DataFrame(input_split_kor["num_kor{}".format(i)].fillna("")).
                pd.DataFrame(input_split_eng["dupl_list_kor{}".format(i)].fillna(""))
            ],
            axis = 1
        )
    print('영문컬럼이 더 많은 싱태')       
    print("kor:" + str(col_cnt_kor) + " vs eng:" + str(col_cnt_eng))
    print("컬럼수 일치작업 후:" + str(col_cnt))  
stwd_eng3

In [ ]:
# n x m 정사각형 DafaFrame 점검
for i in range(col_cnt_kor):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_kor = round(np.mean(avg), 2)
for i in range(col_cnt_eng):
    for j in range(len(globals()["stwd_kor{}".format(i)])):
        cnt = (j + 1)
    avg = np.mean(cnt)
avg_avg_eng = round(np.mean(avg), 2)
stwd_total_cnt = int(avg_avg_eng)
if avg_avg_kor == avg_avg_eng:
    print("한글 영문 row수 일치: 평균=" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))
else:
    yes_or_no("한글명과 영문명의 row count가 일치하지 않습니다. 계속 진행하시겠습니까?")
    print("한글 영문 row수 불일치: 평균" + str((avg_avg_kor + avg_avg_eng)/2))
    print("한글평균:" + str(avg_avg_kor) + ", 영문평균:" + str(avg_avg_eng) + ", 최종count:" + str(stwd_total_cnt))

In [ ]:
# 중복검사를 위한 left join
for i in range(col_cnt):
    globals()['stwd_total{}'.format(i)] = pd.merge(
        globals()['stwd_kor{}'.format(i)],
        globals()['stwd_eng{}'.format(i)],
        on = 'index',
        how = 'left'
    )
print(col_cnt)
# stwd_total9

In [ ]:
# 일렬 세우기

# unique_stwd 초기화 
stwd_total = pd.DataFrame()

## 컬럼명 바꾸고  concat
for i in range(col_cnt):
    globals()["stwd_total{}".format(i)].columns = [
        'index', 'stwd_kor', 'num_kor', 'dupl_list_kor', 'stwd_eng','num_eng', 'dupl_list_eng'
    ]
    stwd_total = pd.concat([stwd_total, globals()["stwd_total{}".format(i)]], axis = 0, ignore_index = True)
print(stwd_total.shape)
stwd_total

# 3. 이상값 수정

### 3.1. 한글명에 한글과 영문대문자, 숫자 이외 표시 및 삭제 로직

In [ ]:
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):

        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")
# 데이터 검토
ans = yes_or_no("데이터 검토를 실행 하시겠습니까?")
if ans == True:
    emoti_chk('kor')
    emoti_chk('eng')
elif ans == False:
    print("계속 진행하겠습니다")

### 3.2. 특수문자 제거 참고

In [ ]:
# 변경 체크 로직

s1 = '고구마acAB1!$%@! 32'
s2 = '깎ㅉ니'
s3 = "RR깍찌니12"
s4 = "123깍찌니aS"
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s1))
print(str(len(re.findall('[A-Z0-9가-힣+]', s1))) + " vs " + str(len(list(s1))))
print(len(re.findall('[A-Z0-9가-힣+]', s1)) == len(list(s1)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s2))
print(str(len(re.findall('[A-Z0-9가-힣+]', s2))) + " vs " + str(len(list(s2))))
print(len(re.findall('[A-Z0-9가-힣+]', s2)) == len(list(s2)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s3))
print(str(len(re.findall('[A-Z0-9가-힣+]', s3))) + " vs " + str(len(list(s3))))
print(len(re.findall('[A-Z0-9가-힣+]', s3)) == len(list(s3)))
print("------------------------------------------------------")
print(re.findall('[A-Z0-9가-힣+]', s4))
print(str(len(re.findall('[A-Z0-9가-힣+]', s4))) + " vs " + str(len(list(s4))))
print(len(re.findall('[A-Z0-9가-힣+]', s4)) == len(list(s4)))
print("------------------------------------------------------")

# 4. 공통 표준 우선 적용

In [ ]:
if yes_or_no("공통 표준으로 통일 하시겠습니까?"):

    # 표준 파일 불러오기
    sheet_name = "공통표준단어"
    input_path = os.path.join(os.getcwd(), "input\\")
    public_stwd = pd.read_excel(input_path + "공공데이터 공통표준용어(2021.12월).xlsx", sheet_name=sheet_name, header=0)
    public_stwd = public_stwd[['NO', '공통표준단어명', '공통표준단어영문약어명']]

    # 표준 영문명 자동 기입(널 제외)

    # 표준 단어명 기준 left join (영문명이 null이면 비표준, notnull이면 표준)
    std_chk = stwd_total.merge(public_stwd, left_on = 'stwd_kor', right_on = '공통표준단어명', how = 'left')
    for i in range(std_chk.shape[0]):
        
        # joined DF의 영문명이 있을 경우 표준으로 대신 기입 
        notna_chk = std_chk['stwd_eng'].notna()[i]
        if notna_chk:
            std_chk.loc[i, 'stwd_eng'] = std_chk.loc[i, '공통표준단어영문약어명']

    std_chk_list = ['index', 'stwd_kor', 'num_kor', 'stwd_eng', 'num_eng']
    std_chk = std_chk[std_chk_list]
    
    print('한글명을 기준으로 공통(행안부) 표준을 우선 적용합니다.')
else:
    print('공통(행안부) 표준을 우선 적용하지 않고 표준화를 진행합니다.')
    
print(std_chk.shape)
std_chk.tail()
std_chk.head()

# 5. 한/영  중복제거 및 통합

In [ ]:
# 한글명 중복 검사: 수기 수정 후 최대 unique 값 카운트
def find_unique_eng(df):
    
    kor_input_cnt = sum(df_step_1['stwd_kor'].notna())
    kor_unique_cnt = len(set(df_step_1['stwd_kor']))
    eng_input_cnt = sum(df_step_1['stwd_eng'].notna())
    eng_unque_cnt = len(set(df_step_1['stwd_eng']))
    
    print("==중복값 비교==")
    # (한)미입력값 개수
    print('(한)     not null   개수: ' + str(kor_input_cnt))
    # (한)unique 값 개수
    print("(한)''포함 unique값 개수: " + str(kor_unique_cnt)) 
    # (영)미입력값 개수
    print('(영)     not null   개수: ' + str(eng_input_cnt))
    # (영)unique 값 개수
    print("(영)''포함 unique값 개수: " + str(eng_unque_cnt)) 
    
    if (kor_input_cnt == eng_input_cnt) and (kor_unique_cnt == eng_unque_cnt):
        print('"한글명과 영문명의 unique 개수가 같습니다."')
    else:
        print('"한글명과 영문명의 unique 개수가 다릅니다."')

In [ ]:
def chk_unique():
    global df_step_1   
    df_step_1 = std_chk
    # 중복제거 후 다시 합쳐서 검사(1안)

    # 한글 중복제거
    col_stwd_kor = ['stwd_kor', 'num_kor']
    unique_kor = df_step_1['stwd_kor'].drop_duplicates()
    stwd_kor_unique = df_step_1.loc[unique_kor.index][col_stwd_kor]
    print("한글명중복제거:"+ str(stwd_kor_unique.shape))
    # stwd_kor_unique

    # 영문 중복제거
    col_stwd_eng = ['stwd_eng', 'num_eng']
    unique_eng = df_step_1['stwd_eng'].drop_duplicates()
    stwd_eng_unique = df_step_1.loc[unique_eng.index][col_stwd_eng]
    print("영문명중복제거:"+ str(stwd_eng_unique.shape))
    # stwd_eng_unique.tail(100)

    # 한글+영문 일렬 세우기
    df_step_1 = pd.DataFrame(stwd_total['index'])
    df_step_1 = df_step_1.merge(
        stwd_kor_unique,
        left_index = True,
        right_index = True,
        how = 'left'
    ).merge(
        stwd_eng_unique,
        left_index = True,
        right_index = True,
        how = 'left'
    )
    print(df_step_1.shape)
    df_step_1

    df_step_1.to_excel(input_path + "df_step_1" + str(dt.datetime.now().strftime('%y%m%d_%H%M%S')) + ".xlsx")
    print(df_step_1.shape)
    df_step_1
    
chk_unique()
find_unique_eng(df_step_1)
df_step_1

# * 한글/영문 단어수 매칭

## 6. 자동수정, 파일 업로드 점검

### 6.1.수기 수정값 표시(! y/n)

In [ ]:
### 값이 한개라도 있을 경우만 카운트 ###

df_step_1_cnt = len(df_step_1)
notna_list = []
for i in range(df_step_1_cnt):    
    # row 기준 널 체크(값이 한개라도 있을 경우 포함)
    if df_step_1.iloc[i].notna().sum() > 1:
        notna_list.append(i)

df_step_1 = df_step_1.loc[notna_list]
print(df_step_1.shape)

# 원래 표준과 병합(수기 수정을 위해)
df_step_1 = df_step_1.merge(
    input_test,
    left_on = 'index',
    right_on = 'No',
    how = 'left'
)

# 영문 개수 세기
dup_list_eng = []
dup_cnt_eng = []
for i in range(df_step_1.shape[0]):
    dup_list_eng = df_step_1['stwd_eng'].to_list().count(df_step_1.loc[i, 'stwd_eng'])
    dup_cnt_eng.append(dup_list_eng)
df_step_1 = pd.concat([df_step_1, pd.DataFrame(dup_cnt_eng, columns = {"eng_cnt"})], axis = 1)

# 컬럼 추리기
df_step_1_list = ['index', 'stwd_eng', 'stwd_kor', '영문약어명', '용어명 분리', 'eng_cnt']
df_step_1 = df_step_1[df_step_1_list]

# 비교군 저장
df_step_1_chk = df_step_1.copy()
df_step_1.head()

# 2차
# 널 제외 한글단어를 표준화된 풀데이터(df_step_1)를 다시검색하여 영문명 자동 기입 : 없는 것은 수동기입 필요 

# 영문 null 값 '' 채우기
df_step_1['stwd_eng'] = df_step_1['stwd_eng'].fillna("")

df_step_1['eng_cor_yn'] = ""
for i in range(df_step_1.shape[0]):
    blnk_chk = df_step_1.loc[i, 'stwd_eng'] == ''
    if blnk_chk :
        df_step_1.loc[i, 'stwd_eng'] = '!' + '_'.join(set(stwd_total.loc[(stwd_total['stwd_kor'] == df_step_1.loc[i, 'stwd_kor']), 'stwd_eng'].to_list()))
        df_step_1.loc[i, 'eng_cor_yn'] = 'n'

df_step_1_chk = df_step_1.copy()
        
# 순서
df_step_1_xlsx = df_step_1.sort_values(['stwd_eng'])
df_step_1_xlsx = df_step_1_xlsx[df_step_1_xlsx['eng_cor_yn'] == 'n']
df_step_1_xlsx.to_excel(output_path+"chk_eng" + ".xlsx")

print(df_step_1_xlsx.shape)
df_step_1_xlsx.head()

### 6.2. 수정 후 재업로드(한글): 개발 미정

### 6.3. 수정 후 재업로드(영문)

#### 6.3.1. 자동수정 후 재업로드(영문)

In [ ]:
# 최초 1회 수정 후 재업로드

# 수기 수정된 데이터
######## 가짜, 추후(chk_eng)로 수정 ########
re_load = pd.read_excel(output_path + 'chk_eng_test.xlsx', index_col='Unnamed: 0')

# 주소값 덮어쓰지 않게 따로 복사하기
df_step_1_cor = df_step_1.copy()

# 모두null일 경우, 그냥 진행, 그렇지 않을 경우 수정된 값 적용 후 진행ㅇ
# Index 병합, 수기수정 대입, 그 외 null 지정
if re_load['stwd_eng'].isna().all():
    pirnt("검수 통과")
else: 
    df_step_1_cor['stwd_eng'] =  re_load['stwd_eng']
    print("수기 수정 다시")
    
# 나머지 null 값은 본래 데이터(df_step_1) 값으로 채워넣기
for i in range(df_step_1_cor.shape[0]):
    if pd.isna(df_step_1_cor.loc[i, 'stwd_eng']):
        df_step_1_cor.loc[i, 'stwd_eng'] = df_step_1.loc[i, 'stwd_eng']
        df_step_1_cor.loc[i, 'eng_cor_yn'] = 'y'

    if df_step_1_cor.loc[i, 'eng_cor_yn'] == 'n':
        if df_step_1_cor.loc[i, 'stwd_eng'] != df_step_1_chk.loc[i, 'stwd_eng']:
            df_step_1_cor.loc[i, 'eng_cor_yn'] = 'y'
        elif df_step_1_cor.loc[i, 'stwd_eng'] == df_step_1_chk.loc[i, 'stwd_eng']:
            df_step_1_cor.loc[i, 'eng_cor_yn'] = 'n'

# 특수문자, 빈칸, null 점검
tmp_list = []
for i in range(df_step_1_cor.shape[0]):
    if len(re.findall('[^A-Z0-9+]', df_step_1_cor.loc[i, 'stwd_eng'])):
        tmp_list.append(i)
    elif df_step_1_cor.loc[i, 'stwd_eng'] == '':
        tmp_list.append(i)
    elif pd.isna(df_step_1_cor.loc[i, 'stwd_eng']):
        tmp_list.append(i)

# 특수문자 'n'으로
df_step_1_cor.loc[tmp_list, 'eng_cor_yn'] = 'n'

# 전체 파일 저장
df_step_1_cor.to_excel(output_path + "df_step_1_cor.xlsx")

# 영문 점검 파일 저장    
chk_eng = df_step_1_cor.loc[df_step_1_cor['eng_cor_yn'] == 'n']
chk_eng.to_excel(output_path + "chk_eng.xlsx")
print(df_step_1_cor.shape)
df_step_1_cor.head()

#### 6.3.2. 수기수정 후 재업로드(영문)

In [ ]:
# 주소값 덮어쓰지 않게 따로 복사하기
df_step_1_cor_tmp = df_step_1_cor.copy()
# 2회 이상부터

def repeat_eng():
    global df_step_1_cor_tmp
    global chk_eng
    global tmp_list_eng
#     global re_load

    # 수기 수정 후 다시 업로드: na_filter=False
    re_load = pd.read_excel(output_path + 'chk_eng.xlsx', index_col='Unnamed: 0', na_filter=False)
    
    # Index 병합, 수기수정 대입, 그 외 ''null'' 지정
    for i in re_load['stwd_eng'].index:
        df_step_1_cor_tmp.loc[i, 'stwd_eng'] =  re_load.loc[i, 'stwd_eng']
    
    # 나머지 null 값은 본래 데이터(df_step_1) 값으로 채워넣기
    for i in range(df_step_1_cor_tmp.shape[0]):
        if df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] == 'n':
            if df_step_1_cor_tmp.loc[i, 'stwd_eng'] != df_step_1_chk.loc[i, 'stwd_eng']:
                df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'y'
            elif df_step_1_cor_tmp.loc[i, 'stwd_eng'] == df_step_1_chk.loc[i, 'stwd_eng']:
                df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'n'
                
        if pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_eng']):
            df_step_1_cor_tmp.loc[i, 'stwd_eng'] = df_step_1_cor.loc[i, 'stwd_eng']
            df_step_1_cor_tmp.loc[i, 'eng_cor_yn'] = 'y'

    # 특수문자, 빈칸, 소문자, 한글, null 점검
    tmp_list_eng = []
    for i in range(df_step_1_cor_tmp.shape[0]):
        if len(re.findall('[^A-Z0-9+]', df_step_1_cor_tmp.loc[i, 'stwd_eng'])):
            tmp_list_eng.append(i)
        elif df_step_1_cor_tmp.loc[i, 'stwd_eng'] == '':
            tmp_list_eng.append(i)
        elif pd.isna(df_step_1_cor_tmp.loc[i, 'stwd_eng']):
            tmp_list_eng.append(i)


    # 특수문자 'n'으로
    df_step_1_cor_tmp.loc[tmp_list_eng, 'eng_cor_yn'] = 'n'

    # 파일 실행
    os.system(output_path + "chk_eng.xlsx")

    # 파일 저장
    chk_eng = df_step_1_cor_tmp.loc[df_step_1_cor_tmp['eng_cor_yn'] == 'n']
    df_step_1_cor_tmp.to_excel(output_path + "df_step_1_cor.xlsx")
    chk_eng.to_excel(output_path + "chk_eng.xlsx")

#### 6.3.3. 재업로드 반복 실행

In [ ]:
# 한글명, 영문명이 모두 ''값일 경우, 수정 완료
if (chk_eng[['stwd_kor', 'stwd_eng']] == '').all().all():
    repeat_eng()
    print('"영문 수정 완료"' + " (빈칸 수:" + str(sum(df_step_1_cor_tmp['stwd_eng'] == '')) + ")")
    os.system(output_path + "df_step_1_cor.xlsx")
else:
    if yes_or_no("영문명 수정이 필요합니다. 반복 점검하시겠습니까?"):
        repeat_eng()
    else:
        os.system(output_path + "chk_eng.xlsx")    